In [ ]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format='retina'

In [ ]:
import os
import json

from tqdm import tqdm
import numpy as np
import scipy.spatial
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import anndata
import scanpy as sc

In [ ]:
import scipy.stats
import statsmodels.stats.multitest

def adjust_p_value_matrix_by_BH(p_val_mtx):
    '''Adjust the p-values in a matrix by the Benjamini/Hochberg method.
    The matrix should be symmetric.
    '''
    p_val_sequential_bh = statsmodels.stats.multitest.multipletests(
        p_val_mtx.reshape(-1), method='fdr_bh')[1]

            
    return p_val_sequential_bh.reshape(p_val_mtx.shape)

In [ ]:
obs_df = pd.read_csv('adata_obs_l2.csv', index_col=0)
print(len(obs_df))

# Keep cells with enough UMI
obs_df = obs_df[(obs_df['num_umis'] > 20)]
obs_df

In [ ]:
# Keep the perturbations with enough cells
pg_counts = obs_df['feature_call'].value_counts()
obs_df = obs_df[obs_df['feature_call'].isin(pg_counts[pg_counts > 20].index)]
obs_df

In [ ]:
output_path = 'enrichment_results'
os.makedirs(output_path, exist_ok=True)
all_clusters = np.unique(obs_df['leiden_l1'])

positive_count_df = pd.DataFrame(
    index=all_clusters,
    columns=np.unique(obs_df['feature_call']),
    dtype=int) 
log2fc_df = pd.DataFrame(
    index=all_clusters,
    columns=np.unique(obs_df['feature_call']),
    dtype=float)
pval_df = log2fc_df.copy()

for cluster_of_interest in log2fc_df.index:
    print(cluster_of_interest)

    for pg_of_interest in tqdm(log2fc_df.columns):
        contigency_table = pd.crosstab(obs_df['leiden_l1'] == cluster_of_interest, 
                            obs_df['feature_call'] == pg_of_interest)
        if contigency_table.shape != (2, 2):
            print(contigency_table.shape)
            continue

        results = scipy.stats.chi2_contingency(contigency_table)
        pval_df.loc[cluster_of_interest, pg_of_interest] = results[1] 

        positive_count_df.loc[cluster_of_interest, pg_of_interest] = contigency_table.values[1, 1]
        log2fc_df.loc[cluster_of_interest, pg_of_interest] = np.log2(
            contigency_table.values[1, 1] / results.expected_freq[1, 1] + 1e-6)

positive_count_df.to_parquet(os.path.join(output_path, 'l1_c_enrich_guide_positive_count.parquet'))
log2fc_df.to_parquet(os.path.join(output_path, 'l1_c_enrich_guide_log2fc.parquet'))
pval_df.to_parquet(os.path.join(output_path, 'l1_c_enrich_guide_pval.parquet'))    

In [ ]:
guide_to_gene_map = {guide : gene for guide, gene in zip(obs_df['feature_call'], obs_df['perturbed_gene'])}

cep_dict = {
    'cluster': [],
    'feature_call': [],
    'perturbed_gene': [],
    'positive_count': [],
    'log2fc': [],
    'pval': []
}

for cluster in positive_count_df.index:
    for pg in positive_count_df.columns:
        cep_dict['cluster'].append(cluster)
        cep_dict['feature_call'].append(pg)
        cep_dict['perturbed_gene'].append(guide_to_gene_map[pg])
        cep_dict['positive_count'].append(positive_count_df.loc[cluster, pg])
        cep_dict['log2fc'].append(log2fc_df.loc[cluster, pg])
        cep_dict['pval'].append(pval_df.loc[cluster, pg])

cep_df = pd.DataFrame(cep_dict)
cep_df['pval_adj'] = statsmodels.stats.multitest.multipletests(
        cep_df['pval'].values, method='fdr_bh')[1]

cep_df.to_parquet(os.path.join(output_path, 'l1_c_enrich_guide.parquet'))

cep_df

In [ ]:
cep_df[(cep_df['log2fc'] > 1) 
     & (cep_df['pval_adj'] < 0.01)
     & (cep_df['positive_count'] > 3)
     & (cep_df['cluster'].isin([15]))
].value_counts('perturbed_gene')

In [ ]:
for cluster in cep_df['cluster'].unique():
    print(cluster)
    pg_counts = cep_df[(cep_df['log2fc'] > 1) 
         & (cep_df['pval_adj'] < 0.01)
         & (cep_df['positive_count'] > 3)
         & (cep_df['cluster'].isin([cluster]))
        ].value_counts('perturbed_gene')
    print(pg_counts.value_counts())
    print()